In [12]:
import numpy as np 
from matplotlib import pyplot as plt 
import seaborn as sns 
from tqdm import tqdm 
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier  , AdaBoostClassifier , GradientBoostingClassifier
from sklearn.metrics import precision_score , recall_score 
import xgboost as xgb 
import pandas as pd 
import sklearn.neighbors._base
from os import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest 
from sklearn.metrics import confusion_matrix , ConfusionMatrixDisplay
sns.set_style('whitegrid')
%load_ext autoreload
%autoreload 2
sns.set_style('whitegrid')
from IPython.display import display

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
def details(data_sent , comments = ''):
    if(comments):
        print(comments)
    sp = (data_sent.isna().sum().sum())/ data_sent.size
    print('________________________________________________')
    print('Sparsity in the data : {:.2f}'.format(sp))
    num_rows = data_sent.shape
    print('Data shape' , num_rows)
    #display(data_sent['class'].value_counts())
    print('Number of sources : ')
    cl = data_sent['class'].unique()
    for c in cl:
        num_src = len(data_sent[data_sent['class']==c]['src_id'].unique())
        num_obs = len(data_sent[data_sent['class']==c])
        print(c ,' \t ' , num_src , '\t' , num_obs)

In [14]:
from features import phot_flux , en_flux , hard, powlaw_fit , bb_fit , brems_fit , intra_obs_var , inter_ob_var , info_col , phot_flux_hilim , phot_flux_lolim , en_flux_hilim , en_flux_lolim

In [15]:
info_col_cl =  info_col+['class']

In [16]:
data_imp_v2 = pd.read_csv('imputed_data/new-data-all-at-once-imp.csv' , index_col='obs_id')

In [17]:
details(data_imp_v2)

________________________________________________
Sparsity in the data : 0.00
Data shape (3103, 59)
Number of sources : 
CV  	  184 	 1582
PL  	  178 	 1000
LX  	  58 	 521


In [18]:
data_id = data_imp_v2[info_col]
data_label = data_imp_v2['class']
data_val = data_imp_v2.drop(columns=['class']+info_col)
data_imp_norm = data_imp_v2.copy()
for c in data_val.columns.to_list():
    data_imp_norm.loc[:,c] = (data_imp_norm.loc[:,c] - data_imp_norm.loc[:,c].mean()) / data_imp_norm.loc[:,c].std()
data_imp_norm

,src_id,num_obs,src_n,name,ra,dec,livetime,significance,likelihood,class,...,kp_prob_b,var_max_b,var_min_b,var_mean_b,var_sigma_b,ks_prob_b,var_prob_b,var_inter_index_s,var_inter_index_m,var_inter_sigma_u
obs_id,,,,,,,,,,,,,,,,,,,,,
CV_0_obs_0,CV_0,25,CSS120424:125906+242634,2CXO J004247.1+411619,10.69654,41.272150,4103.7,52.93,16618.73000,CV,...,-0.026608,-0.049653,-0.309138,-0.300524,0.105078,-0.373526,0.309991,0.083851,-0.812861,0.587816
CV_0_obs_1,CV_0,25,CSS120424:125906+242634,2CXO J004247.1+411619,10.69654,41.272150,3907.5,52.93,16618.73000,CV,...,-0.269488,-0.079118,-0.296220,-0.300223,0.047667,-0.454109,0.148182,0.083851,-0.812861,0.620034
CV_0_obs_2,CV_0,25,CSS120424:125906+242634,2CXO J004247.1+411619,10.69654,41.272150,3907.5,52.93,16618.73000,CV,...,-0.002115,-0.087910,-0.306564,-0.302146,0.007668,-0.380184,0.301590,0.083851,-0.812861,0.595509
CV_0_obs_3,CV_0,25,CSS120424:125906+242634,2CXO J004247.1+411619,10.69654,41.272150,3904.6,52.93,16618.73000,CV,...,-0.173142,-0.272394,-0.294054,-0.315510,-0.141624,-0.382854,0.422968,0.083851,-0.812861,0.586296
CV_0_obs_4,CV_0,25,CSS120424:125906+242634,2CXO J004247.1+411619,10.69654,41.272150,3907.5,52.93,16618.73000,CV,...,0.106439,-0.069993,-0.243589,-0.249620,0.066678,-0.363115,0.202656,0.083851,-0.812861,0.591752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LMXB_86_obs_0,LMXB_86,1,SAX_J1810.8-2609_____,2CXO J181044.4-260901,272.68530,-26.150370,31871.5,9.29,512.97550,LX,...,-0.239104,-0.312200,-0.129559,-0.198625,-0.290714,0.958220,-1.650464,0.150286,-0.828413,-0.145712
LMXB_90_obs_0,LMXB_90,1,XTE_J2123-058________,2CXO J212314.5-054753,320.81060,-5.798128,17705.1,4.06,132.76700,LX,...,-1.510277,-0.311001,-0.352704,-0.387224,-0.230533,-1.224717,1.416856,0.083851,-0.828413,-0.145380
LMXB_91_obs_0,LMXB_91,1,XB_1732-304__________,2CXO J173547.0-302858,263.94590,-30.482820,18855.8,4.89,114.24570,LX,...,-0.208099,-0.350656,-0.312236,-0.349917,-0.242243,0.594397,-0.093470,-1.124061,0.192798,-0.171543


In [19]:
details(data_imp_norm)

________________________________________________
Sparsity in the data : 0.00
Data shape (3103, 59)
Number of sources : 
CV  	  184 	 1582
PL  	  178 	 1000
LX  	  58 	 521


In [20]:
verbose = 0
val_acc = []
test_acc = []
for i in tqdm(range(32)):
    src_list = pd.DataFrame()
    class_to_select = ['CV' , 'LX' , 'PL']
    df = data_imp_norm.copy()
    for cl in class_to_select:
        src = df[df['class']==cl]['src_id'].value_counts().sort_values()
        #display(src)
        src = src.iloc[:int(len(src)*0.4)]
        src_select = src.sample(frac=0.5)
        src_id = src_select.index.to_list()
        src_obs = src_select.to_list()
        temp_src = pd.DataFrame({ 
            'src_id' : src_id , 
            'obs_count' : src_obs
        })
        src_list = src_list.append(temp_src)
    #src_list

    all_src = np.unique(df['src_id'].to_list())
    test_src = src_list['src_id'].to_list()
    train_src= list(set(all_src) - set(src_list['src_id']))
    # if(verbose):
    #     print('Total Number of ssources :' , len(all_src))
    #     print('Training_sources : ' , len(train_src) )
    #     print('Validation sources :' , len(test_src))

    # df = df.replace({
    #     "BH" :2 , 
    #     "NS" : 2 ,
    #     "CV" : 0 , 
    #     "PULSAR" : 1 
    # })


    df= df[df['class'].isin(class_to_select)]
    df 



    df_train  = df[df['src_id'].isin(train_src)]
    df_test = df[df['src_id'].isin(test_src)]

    df_train = df_train[df_train['class'].isin(class_to_select)]
    df_test = df_test[df_test['class'].isin(class_to_select)]

    if(verbose):
        details(df_train)
        details(df_test)

    train_id = df_train[info_col]
    train_label = df_train['class']
    train_value = df_train.drop(columns=['class']+info_col)


    i_val = df_test[info_col]
    y_val = df_test['class']
    x_val = df_test.drop(columns=['class']+info_col)

    clf = RandomForestClassifier(
        n_estimators = 200 , n_jobs=-1
    )
    #clf = AdaBoostClassifier(base_estimator=clf ,)
    #clf = xgb.XGBRFClassifier(n_componets=200)
    x_train , x_test , y_train , y_test = train_test_split(train_value , train_label , test_size=0.2 , stratify=train_label, random_state=9)
    clf.fit(x_train , y_train)
    v_sc = clf.score(x_val , y_val)
    test_sc = clf.score(x_test, y_test)
    val_acc.append(v_sc)
    test_acc.append(test_sc)
    #if(verbose):
    print(v_sc)

  3%|▎         | 1/32 [00:01<00:37,  1.21s/it]

0.5242718446601942


  6%|▋         | 2/32 [00:02<00:33,  1.13s/it]

0.6116504854368932


  9%|▉         | 3/32 [00:03<00:32,  1.12s/it]

...

0.6470588235294118


 97%|█████████▋| 31/32 [00:35<00:01,  1.13s/it]

0.55


100%|██████████| 32/32 [00:36<00:00,  1.13s/it]

0.6213592233009708


In [21]:
score = pd.DataFrame({
    "Test Score" : test_acc, 
    "Validation Score" : val_acc
})*100
score.describe().loc[['mean' , 'std' , 'min', 'max' ]]

,Test Score,Validation Score
mean,94.490537,59.411826
std,0.490502,5.386862
min,93.833333,50.000000
max,95.666667,70.588235
